**Diziden diziye (Seq2Seq) modeller**, bir giriş dizisini alıp bir çıkış dizisi üretebilen yapay sinir ağı modelleridir. Bu modeller genellikle dil çevirisi, metin özeti oluşturma ve konuşma tanıma gibi doğal dil işleme (NLP) problemlerinde kullanılır.

**Temel Bileşenler:**

* **Encoder**: Giriş dizisini kodlayan ve gizli bir vektöre dönüştüren model bileşeni.
* **Decoder**: Gizli vektörü çözerek istenen çıkış dizisini üreten model bileşeni.
* **Attention Mekanizması (opsiyonel)**: Uzun dizilerle çalışırken modelin belirli zaman adımlarında odaklanmasını sağlar.

# RNN (Recurrent Neural Networks)

Tekrar eden bağlantılar içerir ve sıralı verilerle çalışır.
Uzun dizilerde gradyan patlaması veya kaybolması problemi yaşar.

**Matematiksel Gösterim:**

$$h_t = \sigma(W_h \cdot h_{t-1} + W_x \cdot x_t + b)$$



![Recurrent Neural Network](pictures/RNN.jpeg "Recurrent Neural Network")

# LSTM (Long Short-Term Memory)

Bellek hücreleri ve kapılar (gate) kullanarak uzun süreli bağımlılıkları öğrenir.

* Kapılar:
* Forget Gate: Önceki bilgiyi unutur.
* Input Gate: Yeni bilgiyi ekler.
* Output Gate: Çıkışı üretir.

**Matematiksel Gösterim:**
    
1. **Forget Gate:**

   $$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$$

2. **Input Gate:**

   $$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$$

3. **Candidate Memory Cell:**

   $$\tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)$$

4. **Memory Cell Update:**

   $$C_t = f_t \ast C_{t-1} + i_t \ast \tilde{C}_t$$

5. **Output Gate:**

   $$h_t = o_t \ast \tanh(C_t)$$


![Long Short-Term Memory](pictures/LSTM.jpeg "Long Short-Term Memory")

# GRU (Gated Recurrent Unit)

LSTM’nin daha sade bir versiyonudur. Daha az parametre içerir.

**Kapılar:**

**Reset Gate**: Önceki bilgiyi sıfırlar.

**Update Gate**: Mevcut bilgiyi günceller

Bir Seq2Seq Modeli Oluşturma

Basit bir seq2seq modeli oluşturma

Başlamadan önce bir conda environment kuralım ve paket yüklemelerini bu environment içinde yapalım.

Anaconda Prompt'u açınız ve sırasıyla şu komutları çalıştırınız:

* conda create -n tensorflow_env_3-9 python=3.9
* conda activate tensorflow_env_3-9
* conda install tensorflow numpy scipy
* pip install notebook ipykernel

Şimdi python dosyanızın bulunduğu konuma giderek jupyter notebook'u çalıştırınız.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense

# Encoder
encoder_inputs = Input(shape=(None, 128))  # 128 boyutlu giriş
encoder = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None, 128))
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(128, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()


# Rastgele veri oluşturma
encoder_input_data = np.random.rand(100, 10, 128)  # 100 örnek, her biri 10 zaman adımı, 128 boyutlu vektör
decoder_input_data = np.random.rand(100, 10, 128)
decoder_target_data = np.random.rand(100, 10, 128)

# Modeli eğitme
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=32, epochs=10)


# Encoder Modeli
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder Modeli
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Çıkış Üretimi
states_value = encoder_model.predict(encoder_input_data[:1])
decoder_input = np.zeros((1, 1, 128))  # Başlangıç sembolü
decoded_sequence = []

for _ in range(10):  # Maksimum 10 zaman adımı
    output_tokens, h, c = decoder_model.predict([decoder_input] + states_value)
    decoded_sequence.append(output_tokens)
    decoder_input = output_tokens  # Bir sonraki zaman adımının girişi
    states_value = [h, c]

    


Sentiment Analiz Uygulaması

In [ ]:
!pip install pandas==1.5.3
!pip install scikit-learn==1.1.3

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# 1. Veri Yükleme
file_path = "dosyalar/twitter_training.csv"  # CSV dosyasının yolu
data = pd.read_csv(file_path, header=None, names=["ID", "Category", "Sentiment", "Text"])

# 2. Veriyi İşleme
# Sadece metin (Text) ve duygu (Sentiment) sütunlarını alıyoruz
texts = data["Text"].values
labels = data["Sentiment"].values

# Tüm metinlerin string formatında olduğundan emin olalım
texts = [str(text) if not isinstance(text, str) else text for text in texts]

# Sentiment'i sayısal değerlere dönüştürme (Positive -> 1, Negative -> 0, Neutral -> 2)
label_dict = {"Positive": 1, "Negative": 0, "Neutral": 2}
labels = np.array([label_dict.get(label, -1) for label in labels])

# Geçersiz etiketleri temizleme
valid_labels = set(label_dict.values())  # Geçerli etiketler: {0, 1, 2}
filtered_indices = [i for i, label in enumerate(labels) if label in valid_labels]
texts = [texts[i] for i in filtered_indices]
labels = [labels[i] for i in filtered_indices]

# 3. Metni Sayısal Hale Getirme (Tokenization)
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index

# Metinleri dizilere dönüştürme
sequences = tokenizer.texts_to_sequences(texts)

# Dizileri sabit uzunlukta padding ile doldurma
max_length = max(len(seq) for seq in sequences)  # En uzun diziyi referans al
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post")

# 4. Eğitim ve Test Verilerini Ayırma
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Etiketleri numpy array'e dönüştürme
y_train = np.array(y_train)
y_test = np.array(y_test)

# 5. Model Oluşturma
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=max_length),  # Embedding katmanı
    LSTM(128, return_sequences=False),  # LSTM katmanı
    Dropout(0.5),  # Overfitting'i önlemek için dropout
    Dense(64, activation="relu"),
    Dense(3, activation="softmax")  # Çok sınıflı sınıflandırma için softmax
])

# Modeli derleme
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

# 6. Modeli Eğitme
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test)
)

# 7. Modelin Test Edilmesi
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# 8. Yeni Metinlerin Tahmini
new_texts = ["I will destroy you in Borderlands!", "I am enjoying Borderlands 2."]
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_length, padding="post")
predictions = model.predict(new_padded_sequences)

for text, pred in zip(new_texts, predictions):
    sentiment = ["Negative", "Positive", "Neutral"][np.argmax(pred)]
    print(f"Text: {text} -> Sentiment: {sentiment}")


In [ ]:
import os
import re
import numpy as np
import tensorflow as tf
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# 1. Önişleme Fonksiyonları
def extract_text_from_xml(file_path):
    """XML dosyasından metni çıkarır."""
    try:
        import xml.etree.ElementTree as ET
        tree = ET.parse(file_path)
        root = tree.getroot()
        text_field = root.find(".//field[@name='text']")
        if text_field is not None:
            return text_field.text.strip()
        else:
            return ""
    except ET.ParseError as e:
        print(f"Hata: {file_path} XML dosyası düzgün değil. Hata: {e}")
        return ""

def preprocess_text(text):
    """Metin önişleme işlemleri."""
    text = text.lower()  # Küçük harfe çevirme
    text = re.sub(r'\d+', '', text)  # Sayıları kaldırma
    text = re.sub(r'[^\w\s]', '', text)  # Noktalama işaretlerini kaldırma
    return text

def load_documents_and_labels(base_dir):
    """Belge ve etiketleri klasör yapısından yükler."""
    categories = os.listdir(base_dir)
    documents = []
    labels = []
    for category in categories:
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path):
            for file_name in os.listdir(category_path):
                if file_name.endswith(".xml"):
                    file_path = os.path.join(category_path, file_name)
                    text = extract_text_from_xml(file_path)
                    processed_text = preprocess_text(text)
                    documents.append(processed_text)
                    labels.append(category)
    return documents, labels

# 2. Kelime Vektörlerini Yükleme
def load_custom_word_vectors(file_path):
    """Kelime vektörlerini dosyadan yükler."""
    word_vectors = {}
    vector_size = None
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split('\t')
            word = parts[0]
            vector = np.array([float(x) for x in parts[1:]])
            if vector_size is None:
                vector_size = len(vector)
            if len(vector) == vector_size:
                word_vectors[word] = vector
    return word_vectors

def create_embedding_matrix(word_vectors, word_index, vector_size):
    """Embedding matrisi oluşturur."""
    embedding_matrix = np.zeros((len(word_index) + 1, vector_size))
    for word, i in word_index.items():
        if word in word_vectors:
            embedding_matrix[i] = word_vectors[word]
    return embedding_matrix

# 3. Veri Yükleme
base_dir = 'dosyalar/Milliyet-Collection-Mini'
documents, labels = load_documents_and_labels(base_dir)

# 4. Etiketleri Kodlama
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

# 5. Metinleri Tokenize Etme
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(documents)
X = tokenizer.texts_to_sequences(documents)
max_sequence_length = max(len(seq) for seq in X)
X = pad_sequences(X, maxlen=max_sequence_length)

# 6. Kelime Vektörlerini Yükleme ve Embedding Matrisi Oluşturma
word_vectors = load_custom_word_vectors('dosyalar/cbow_word_vectors.txt')  # Kelime vektörleri dosyası
vector_size = len(next(iter(word_vectors.values())))  # İlk vektörün boyutunu al
embedding_matrix = create_embedding_matrix(word_vectors, tokenizer.word_index, vector_size)

# 7. LSTM Modeli
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, 
              output_dim=vector_size, 
              weights=[embedding_matrix], 
              input_length=max_sequence_length, 
              trainable=False),  # Önceden eğitilmiş vektörleri sabitle
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y)), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 8. Eğitim ve Test Setlerine Bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 9. Modeli Eğitme
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# 10. Modeli Değerlendirme
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Doğruluk (Accuracy): {accuracy:.2f}")


In [ ]:
def predict_category(text, model, tokenizer, label_encoder, max_sequence_length):
    # 1. Önişleme
    processed_text = preprocess_text(text)
    
    # 2. Sekansa Dönüştürme
    sequence = tokenizer.texts_to_sequences([processed_text])  # Tek metin için
    padded_sequence = pad_sequences(sequence, maxlen=max_sequence_length)
    
    # 3. Tahmin Yapma
    prediction = model.predict(padded_sequence)
    predicted_label_index = np.argmax(prediction, axis=1)  # En yüksek olasılığa sahip sınıfı al
    
    # 4. Tahmini Etiketle Dönüştürme
    predicted_label = label_encoder.inverse_transform(predicted_label_index)
    
    return predicted_label[0]

# Örnek metin
example_text = "Merkez bankası para arzını artıracak."

# Kategori tahmini
predicted_category = predict_category(example_text, model, tokenizer, label_encoder, max_sequence_length)
print(f"Tahmin edilen kategori: {predicted_category}")
